In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
#The file names
file_predicted = "../insight_testsuite/tests/test_2/input/predicted.txt"
file_actual = "../insight_testsuite/tests/test_2/input/actual.txt"
file_window = "../insight_testsuite/tests/test_2/input/window.txt"

In [3]:
def parse_data(filename, names=['hour', 'stock_id', 'predicted']):
    
    df = pd.read_csv(filename, sep="|", header=None, names=names)
    df.sort_values(by=['hour','stock_id'])
    
    col = [str(hour)+stock_id for hour,stock_id in zip(list(df['hour']), list(df['stock_id']))]
    df.index = col
    
    return df

def compute_errors(df, window):
    
    min_time = min(sub.hour)
    max_time = max(sub.hour)
    window 
    
    results = {}
    for time in range(min_time, (max_time-window+1)+1):
        
        end_time = time + window - 1
        ibool = (df['hour'] >= time) & (df['hour'] <= end_time)
        idf = df[ibool]
        err = abs(idf['actual']-idf['predicted']).mean()
        
        results[time] = [time, end_time, err]
        
    results = pd.DataFrame(results).T
    results.columns = ['start_hour', 'end_hour', 'error']
    
    return results

def compute_error(df, window):
    
    min_time = min(df.hour)
    max_time = max(df.hour) 
    
    results = {}
    for time in range(min_time, (max_time-window+1)+1):
        
        end_time = time + window - 1
        ibool = (df['hour'] >= time) & (df['hour'] <= end_time)
        idf = df[ibool]
        #err = abs(idf['actual']-idf['predicted']).mean()
        err = abs(idf.error).mean()
        
        results[time] = [time, end_time, err]
        
    results = pd.DataFrame(results).T
    results.columns = ['start_hour', 'end_hour', 'error']
    
    return results

In [4]:
predicted = parse_data(file_predicted)
predicted.head()

,hour,stock_id,predicted
1AMDDPW,1,AMDDPW,23.46
1CCKENL,1,CCKENL,25.27
1NELVVI,1,NELVVI,22.41
1LWZQMJ,1,LWZQMJ,19.14
1ZMGTBK,1,ZMGTBK,23.39


In [15]:
actual = parse_data(file_actual, names=['hour', 'stock_id', 'actual'])
actual.head()

if ('hour' in actual) | ('stock_id' in actual) : print(0)

0


In [6]:
window = int(open(file_window,"r").readlines()[0])
print(window)

4


In [7]:
sub = actual.loc[predicted.index] 
print(sub.shape, actual.shape, predicted.shape)
#print(sub.actual == predicted.predicted)
print(sum(sub.index != predicted.index))
sub.head()

(114194, 3) (142560, 3) (114194, 3)
0


,hour,stock_id,actual
1AMDDPW,1,AMDDPW,23.46
1CCKENL,1,CCKENL,25.27
1NELVVI,1,NELVVI,22.41
1LWZQMJ,1,LWZQMJ,19.14
1ZMGTBK,1,ZMGTBK,23.39


In [8]:
sub['predicted'] = predicted.predicted
sub['error'] = abs(sub.actual - sub.predicted)
sub.head()

,hour,stock_id,actual,predicted,error
1AMDDPW,1,AMDDPW,23.46,23.46,0.0
1CCKENL,1,CCKENL,25.27,25.27,0.0
1NELVVI,1,NELVVI,22.41,22.41,0.0
1LWZQMJ,1,LWZQMJ,19.14,19.14,0.0
1ZMGTBK,1,ZMGTBK,23.39,23.39,0.0


In [9]:
res = compute_error(sub, window=window)
res['start_hour'] = res['start_hour'].map(lambda x: '%d'%(x)) 
res['end_hour'] = res['end_hour'].map(lambda x: '%d'%(x))
pd.DataFrame(res).to_csv('../comparison1.txt', sep='|', index=False, header=False, float_format='%3.2f')
pd.DataFrame(sub).to_csv('../out.txt', sep=' ', index=False, header=False, float_format='%3.2f')
res.head()

,start_hour,end_hour,error
1,1,4,0.000102
2,2,5,0.000131
3,3,6,0.000191
4,4,7,0.000381
5,5,8,0.000521


In [10]:
merged_result = pd.merge(actual, predicted, on=['hour', 'stock_id']) 
print(merged_result.shape)
merged_result.head()

(114194, 4)


,hour,stock_id,actual,predicted
0,1,AMDDPW,23.46,23.46
1,1,CCKENL,25.27,25.27
2,1,NELVVI,22.41,22.41
3,1,LWZQMJ,19.14,19.14
4,1,ZMGTBK,23.39,23.39


In [11]:
#get maximum time
min_time = min(sub.hour)
max_time = max(sub.hour)
print(min_time, max_time)

1 1440


In [12]:
results = compute_errors(merged_result, window=window)

In [13]:
results['start_hour'] = results['start_hour'].map(lambda x: '%d'%(x)) 
results['end_hour'] = results['end_hour'].map(lambda x: '%d'%(x))
pd.DataFrame(results).to_csv('../comparison2.txt', sep='|', index=False, header=False, float_format='%3.2f')